In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt 
import scipy.spatial.distance as dis
import random as rd
import imageio

In [ ]:
def kernel(S,C,gamma1,gamma2):
    '''
    Impletation of kernel
    S: (n,2)-array, spatial information of data
    C: (n,3)-array, color information of data
    gamma1, gamma2: parameters
    return a (n,n)-array
    ''' 
    RBFS=dis.squareform(np.exp(-gamma1*dis.pdist(S,'sqeuclidean')))
    RBFC=dis.squareform(np.exp(-gamma2*dis.pdist(C,'sqeuclidean')))
    return RBFS*RBFC+np.identity(np.size(S,0))

In [ ]:
img1=cv2.imread('image1.png')
img2=cv2.imread('image2.png')
# spatial information (x_1,x_2)
X=np.zeros((10000,2))
for i in range(10000):
    X[i][0]=i//100+1
    X[i][1]=i%100+1
# color information (y_1,y_2,y_3)
Y1=img1.reshape((10000,3))
Y2=img2.reshape((10000,3))
# kernel kmeans
K1=kernel(X,Y1,0.001,0.001)
K2=kernel(X,Y2,0.001,0.001)

# Initialization of kmeans

Randomized

In [ ]:
def init_list1(k):
    '''
    k: number of clusters
    return the list containing index of k centers
    '''
    list1=[]
    for i in range(k):
        x=rd.randint(0,9999)
        list1.append(x)
    return list1

kmeans++

In [ ]:
def init_list2(Mat,k):
    '''
    Mat: (n,n)-array, weight of all pairs of data
    k: number of clusters
    return the list containing index of k centers
    '''
    list1=[]
    weight=np.zeros(10000)
    j=rd.randint(0,9999)
    list1.append(j)
    weight+=Mat[j]
    for i in range(k-1):
        prob=np.cumsum(weight/np.sum(weight))
        x=rd.random()
        for j in range(10000):
            if x<prob[j]:
                list1.append(j)
                break
        dis=np.zeros((2,10000))
        dis[0]=weight
        dis[1]=Mat[j]
        weight=np.min(dis,axis=0)
    return list1

# kmeans clustering

In [ ]:
def clustering(list1,K,k):
    '''
    list1: initial centers of every cluster
    K: kernel function
    k: number of clusters
    return (n,)-array, the index of data
    '''
    cluster=np.zeros((100,100))
    alpha=np.zeros((k,10000))
    for i in range(k):
        alpha[i][list1[i]]=1
    for t in range(10):
        dist=np.ones((10000,k))
        mean=np.zeros(k)
        for i in range(k):
            mean[i]=1/np.sum(alpha[i])**2*np.sum(np.outer(alpha[i],alpha[i])*K)
        for i in range(10000):
            for j in range(k):
                dist[i,j]-=2/np.sum(alpha[j])*np.sum(alpha[j]*K[i])
                dist[i,j]+=mean[j]        
        for i in range(10000):
            ind=np.argmin(dist[i])
            cluster[i//100][i%100]=ind
            for j in range(k):
                alpha[j][i]=0
            alpha[ind][i]=1
        plt.imsave('new_'+str(t)+'.png',cluster)
    # Gif
    imagelist=[]
    for i in range(10):
        imagelist.append('new_'+str(i)+'.png')
    with imageio.get_writer('newgif.gif', mode='I') as writer:
        for filename in imagelist:
            image = imageio.imread(filename)
            writer.append_data(image)
    return cluster.reshape(1,-1)

# kmeans clustering (test)

In [ ]:
kmeans2=init_list2(np.log(1/K1),2)
clustering(kmeans2,K1,2)

In [ ]:
kmeans3=init_list2(np.log(1/K1),3)
clustering(kmeans3,K1,3)

In [ ]:
kmeans4=init_list2(np.log(1/K1),4)
clustering(kmeans4,K1,4)

In [ ]:
kmeans2=init_list2(np.log(1/K2),2)
clustering(kmeans2,K2,2)

In [ ]:
kmeans3=init_list2(np.log(1/K2),3)
clustering(kmeans3,K2,3)

In [ ]:
kmeans4=init_list2(np.log(1/K2),4)
clustering(kmeans4,K2,4)

# Spectral clustering

Normalized cut

In [ ]:
def normal_cut(K):
    '''
    K: kernel of data
    return (n)-array, eigenvalue and (n,n)-array, eigenvector
    '''
    D=np.diag(np.sum(K,axis=1))
    L=D-K
    D_inverse_square_root=np.diag(1/np.diag(np.sqrt(D)))
    L_sym=D_inverse_square_root@L@D_inverse_square_root
    eigenvalue,eigenvector=np.linalg.eigh(L_sym)
    return eigenvalue,eigenvector

Ratio cut

In [ ]:
def ratio_cut(K):
    '''
    K: kernel of data
    return (n)-array, eigenvalue and (n,n)-array, eigenvector
    '''
    D=np.diag(np.sum(K,axis=1))
    L=D-K
    eigenvalue,eigenvector=np.linalg.eigh(L)
    return eigenvalue,eigenvector

# Spectral clustering (test)

In [ ]:
eigenvaluen1,eigenvectorn1=normal_cut(K1)
sort_indexn1=np.argsort(eigenvaluen1)

In [ ]:
k=2
T=eigenvectorn1[:,sort_indexn1[1:k+1]]
sums=np.sqrt(np.sum(np.square(T),axis=1)).reshape(-1,1)
H=T/sums
Hdist=dis.squareform(dis.pdist(H,'sqeuclidean'))
normal=init_list2(Hdist,k)
A1=clustering(normal,H@H.T,k)

In [ ]:
eigenvaluen2,eigenvectorn2=normal_cut(K2)
sort_indexn2=np.argsort(eigenvaluen2)

In [ ]:
k=2
T=eigenvectorn2[:,sort_indexn2[1:k+1]]
sums=np.sqrt(np.sum(np.square(T),axis=1)).reshape(-1,1)
H=T/sums
Hdist=dis.squareform(dis.pdist(H,'sqeuclidean'))
normal=init_list2(Hdist,k)
A2=clustering(normal,H@H.T,k)

In [ ]:
eigenvaluer1,eigenvectorr1=ratio_cut(K1)
sort_indexr1=np.argsort(eigenvaluer1)

In [ ]:
k=2
H=eigenvectorr1[:,sort_indexr1[1:k+1]]
Hdist=dis.squareform(dis.pdist(H,'sqeuclidean'))
ratio=init_list2(Hdist,k)
A3=clustering(ratio,H@H.T,k)

In [ ]:
eigenvaluer2,eigenvectorr2=ratio_cut(K2)
sort_indexr2=np.argsort(eigenvaluer2)

In [ ]:
k=2
H=eigenvectorr2[:,sort_indexr2[1:k+1]]
Hdist=dis.squareform(dis.pdist(H,'sqeuclidean'))
ratio=init_list2(Hdist,k)
A4=clustering(ratio,H@H.T,k)

# Part 4

In [ ]:
def plot_eigenspace(x_axis,y_axis,class_cluster):
    plt.figure()
    colors=['purple','orange']
    for colors,i in zip(colors,np.arange(2)):
        plt.scatter(x_axis[class_cluster==i],y_axis[class_cluster==i],c=colors,s=3)
    plt.xlabel('$1^{st}$ Eigenvector')
    plt.ylabel('$2^{nd}$ Eigenvector')
    plt.show()

In [ ]:
x_a=T[:,0]
y_a=T[:,1]
plot_eigenspace(x_a.reshape(1,-1),y_a.reshape(1,-1),A1.astype(int))

In [ ]:
x_a=H[:,0]
y_a=H[:,1]
plot_eigenspace(x_a.reshape(1,-1),y_a.reshape(1,-1),A2.astype(int))

In [ ]:
x_a=H[:,0]
y_a=H[:,1]
plot_eigenspace(x_a.reshape(1,-1),y_a.reshape(1,-1),A3.astype(int))

In [ ]:
x_a=H[:,0]
y_a=H[:,1]
plot_eigenspace(x_a.reshape(1,-1),y_a.reshape(1,-1),A4.astype(int))